# HHL 

Try to implement HHL.

## imports

In [809]:
import numpy as np
import scipy as sp
# Import Qiskit
from qiskit import QuantumCircuit, transpile,  QuantumRegister, ClassicalRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import Operator, Statevector

from qiskit.circuit.library.phase_estimation import PhaseEstimation
from qiskit.circuit.library.arithmetic.exact_reciprocal import ExactReciprocal

## Parameters

In [810]:
nb = 1
nl = 2 # eigenvalue evaluation qubits
nf = 1 # if state preparation is probabilistic the number of qubit flags should increase
# na = 0 # ancillia count
t = np.pi * 3/4
c = 1/8
A = np.array([[1,0],[0,1]]) 
b_ = np.array([7,1]) 
b = b_ / np.linalg.norm(b_)

## Prepare Circuit

In [811]:
# Create a new circuit with two qubits
rb = QuantumRegister(nb)
rl = QuantumRegister(nl)
rf = QuantumRegister(nf)
# crf = ClassicalRegister(nf)
# crb = ClassicalRegister(nb)
# ra = QuantumRegister(na)

qc = QuantumCircuit(rb,rl,rf)#,crf,crb)

## State Preparation

In [812]:
qcb = QuantumCircuit(rb)
qcb.prepare_state(b / np.linalg.norm(b))

qc.append(qcb, rb[:])

# display(qc.draw())

psi = Statevector(qc)
psi.draw("latex")

<IPython.core.display.Latex object>

# QPE

In [813]:
qcA = QuantumCircuit(rb)
eA = sp.linalg.expm(1j * A * t)
qcA.append(Operator(eA).to_instruction(),[0])

qcpe = PhaseEstimation(nl,qcA)
qc.append(qcpe, rl[:] + rb[:])

psi = Statevector(qc)
psi.draw("latex")

<IPython.core.display.Latex object>

## Controled Rotation

In [814]:
qccr = ExactReciprocal(nl,c)

qc.append(qccr.to_instruction(),  rl[::-1] + [rf[0]])
# qc.draw()
psi = Statevector(qc)
psi.draw("latex")

<IPython.core.display.Latex object>

## QPE inverse

In [815]:
qc.append(qcpe.inverse(), rl[:] + rb[:])

## Show Result

In [816]:
psi = Statevector(qc)
psi.draw("latex")

<IPython.core.display.Latex object>

## Simulation

In [817]:
# Transpile for simulator
simulator = AerSimulator()
qc = transpile(qc, simulator)

# qc.measure(rf,crf)
# qc.measure(rb,crb)

# Run and get counts
result = simulator.run(qc).result()
counts = result.get_counts(qc)
counts_filiter_1 = {k[2]: v for k,v in counts.items() if k.startswith('1')}
plot_histogram(counts_filiter_1, title='State counts')



QiskitError: 'No counts for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000020948F574D0>"'